In [70]:
!conda install pandas -y

Jupyter detected...
2 channel Terms of Service accepted
Channels:
 - defaults
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.



In [71]:
# Downloading ECB rates from URLs

import urllib.request as request_url
import pandas as pd

daily_rates_URL = "https://www.ecb.europa.eu/stats/eurofxref/eurofxref.zip"
historical_rates_URL = "https://www.ecb.europa.eu/stats/eurofxref/eurofxref-hist.zip"


request_url.urlretrieve(daily_rates_URL, "eurofxref.zip")
request_url.urlretrieve(historical_rates_URL, "eurofxref-hist.zip")

('eurofxref-hist.zip', <http.client.HTTPMessage at 0x10c85f050>)

In [72]:
# Unzipping zip files

!unzip eurofxref.zip -d eurofxref
!unzip eurofxref-hist.zip -d eurofxref-hist

Archive:  eurofxref.zip
  inflating: eurofxref/eurofxref.csv  
Archive:  eurofxref-hist.zip
  inflating: eurofxref-hist/eurofxref-hist.csv  


In [73]:
# Reading eurofxref.csv file to pandas DataFrame

df = pd.read_csv('eurofxref/eurofxref.csv')
df.head()

,Date,USD,JPY,CZK,DKK,GBP,HUF,PLN,RON,SEK,...,INR,KRW,MXN,MYR,NZD,PHP,SGD,THB,ZAR,
0,09 February 2026,1.1886,185.65,24.217,7.4692,0.8701,377.08,4.2118,5.0923,10.6505,...,107.8375,1737.79,20.4582,4.6765,1.9685,69.516,1.5067,37.055,18.9883,


In [74]:
# Reading eurofxref-hist.csv file to pandas DataFrame

df_history = pd.read_csv('eurofxref-hist/eurofxref-hist.csv')
df_history.head()

,Date,USD,JPY,BGN,CYP,CZK,DKK,EEK,GBP,HUF,...,INR,KRW,MXN,MYR,NZD,PHP,SGD,THB,ZAR,Unnamed: 42
0,2026-02-09,1.1886,185.65,NaN,NaN,24.217,7.4692,NaN,0.8701,377.08,...,107.8375,1737.79,20.4582,4.6765,1.9685,69.516,1.5067,37.055,18.9883,NaN
1,2026-02-06,1.1794,185.27,NaN,NaN,24.240,7.4672,NaN,0.8679,377.95,...,106.9265,1730.68,20.4787,4.6557,1.9673,69.047,1.5016,37.269,19.0170,NaN
2,2026-02-05,1.1798,185.11,NaN,NaN,24.310,7.4662,NaN,0.8691,379.65,...,106.5460,1728.58,20.5112,4.6567,1.9676,69.299,1.5021,37.488,19.0889,NaN
3,2026-02-04,1.1820,185.15,NaN,NaN,24.357,7.4673,NaN,0.8616,380.38,...,106.8698,1720.70,20.3808,4.6476,1.9602,69.721,1.5027,37.390,18.8381,NaN
4,2026-02-03,1.1801,183.92,NaN,NaN,24.312,7.4687,NaN,0.8623,380.40,...,106.3710,1709.44,20.4245,4.6407,1.9532,69.732,1.4994,37.250,18.8218,NaN


In [75]:
# Outside help used for dropping columns: https://stackoverflow.com/questions/56891518/drop-columns-from-pandas-dataframe-if-they-are-not-in-specific-list


# Dropping all columns except selected_currencies
# eurofxref.csv had whitespace before column names, so needed stripping

selected_currencies = ["USD", "SEK", "GBP", "JPY"]

df.columns = df.columns.str.strip()

df_help = df[df.columns.intersection(selected_currencies)]
df_help.head()

,USD,JPY,GBP,SEK
0,1.1886,185.65,0.8701,10.6505


In [76]:
# Dropping all columns except selected_currencies

df_history = df_history[df_history.columns.intersection(selected_currencies)]
df_history.head()

,USD,JPY,GBP,SEK
0,1.1886,185.65,0.8701,10.6505
1,1.1794,185.27,0.8679,10.6735
2,1.1798,185.11,0.8691,10.6410
3,1.1820,185.15,0.8616,10.5770
4,1.1801,183.92,0.8623,10.5485


In [77]:
# Table with needed columns to export data to html

to_table = pd.DataFrame(columns=["Currency Code", "Rate", "Mean Historical Rate"])

In [78]:
# Getting mean historical rate and adding data to 'to_table'

for column in df_history.columns:
  total = 0
  for row in df_history[column]:
    total += row
  avg = total / df_history[column].shape[0]
  to_table.loc[len(to_table.index)] = [column, df[column][0], avg]
  print(column, avg)

USD 1.182286729106629
JPY 131.59171613832868
GBP 0.7847394495677198
SEK 9.688798465417863


In [79]:
# Controlling if the data is as needed

to_table.head()

,Currency Code,Rate,Mean Historical Rate
0,USD,1.1886,1.182287
1,JPY,185.6500,131.591716
2,GBP,0.8701,0.784739
3,SEK,10.6505,9.688798


In [80]:
# DataFrame to html without indexes

to_table.to_html("exchange_rates.html", index=False)